In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer
import hummingbird

#### Create and fit the model

In [15]:
# Create and train a RandomForestClassifier model
X, y = load_breast_cancer(return_X_y=True)
skl_model = RandomForestClassifier(n_estimators=500, max_depth=7)
skl_model.fit(X, y)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=7, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Time scikit-learn

In [16]:
%%timeit
# Execute prediction using scikit-learn model
pred = skl_model.predict(X)

59.2 ms ± 234 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Convert SKL model to PyTorch

In [17]:
# Use Hummingbird to convert the model to pytorch
model = skl_model.to('pytorch')

#### Time PyTorch - CPU

In [18]:
%%timeit
pred_cpu_hb = model.predict(X)

10.1 ms ± 169 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Switch PyTorch from CPU to GPU

In [19]:
%%capture 
model.to('cuda')

#### Time PyTorch - GPU

In [20]:
%%timeit
pred_gpu_hb = model.predict(X)


1.18 ms ± 8.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
